In [3]:
# 7.2.1 - 인간과 AI 간 메시지 교대 (ChatPromptTemplate)
# 채팅 기반 애플리케이션에서 시스템, 인간, AI 메시지를 구조화하는 방법

# 필요한 라이브러리 설치
# !pip install langchain==0.2.17 langchain-openai

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    if api_key:
        os.environ['OPENAI_API_KEY'] = api_key
        print("API 키가 설정되었습니다!")
else:
    print("기존 환경 변수의 API 키를 사용합니다.")

# =============================================================================
# 라이브러리 import
# =============================================================================
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

# OpenAI Chat 모델 초기화
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

print("=" * 80)
print("7.2.1 ChatPromptTemplate을 이용한 해적 번역기")
print("=" * 80)

print("""
원서 코드의 문제점:
1. LLMChain 클래스가 deprecated됨
2. chain.run() 메소드가 deprecated됨

수정 내용:
- LLMChain → RunnableSequence (prompt | llm)
- chain.run() → chain.invoke()

핵심 개념 - ChatPromptTemplate:
- 시스템 메시지: AI의 역할과 행동 방식 정의
- 예시 메시지: few-shot learning을 위한 대화 예시
- 인간 메시지: 사용자 입력 템플릿
""")

# =============================================================================
# ChatPromptTemplate 구성 요소들
# =============================================================================

# 1. 시스템 역할 정의 (AI의 페르소나 설정)
template = "You are a helpful assistant that translates English to pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# 2. Few-shot 예시 정의 (인간-AI 대화 예시)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")

# 3. 유저 입력 프롬프트 정의
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 4. 전체 ChatPromptTemplate 구성
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,  # 시스템 역할
    example_human,          # 예시 - 인간 메시지
    example_ai,            # 예시 - AI 응답
    human_message_prompt   # 실제 사용자 입력
])

print("\n생성된 프롬프트 구조:")
sample_messages = chat_prompt.format_messages(text="I love programming.")
for i, message in enumerate(sample_messages):
    print(f"{i+1}. {message.__class__.__name__}: {message.content}")

# =============================================================================
# 최신 방식으로 실행
# =============================================================================
print("\n" + "=" * 60)
print("실행 결과")
print("=" * 60)

# 최신 방식: RunnableSequence
chain = chat_prompt | chat

# 테스트 문장들
test_sentences = [
    "I love programming.",
    "How are you today?",
    "The weather is beautiful.",
    "Can you help me with my homework?",
    "Good morning!"
]

try:
    print("해적 번역기 결과:")
    print("-" * 40)
    
    for sentence in test_sentences:
        response = chain.invoke({"text": sentence})
        print(f"영어: {sentence}")
        print(f"해적어: {response.content}")
        print()
        
except Exception as e:
    print(f"오류 발생: {e}")

# =============================================================================
# 기존 방식과 비교
# =============================================================================
print("=" * 60)
print("기존 방식 vs 새로운 방식")
print("=" * 60)

print("""
원서 코드 (Deprecated):
from langchain.chains import LLMChain

chain = LLMChain(llm=chat, prompt=chat_prompt)
response = chain.run({"text": "I love programming."})

수정된 코드 (현재 권장):
chain = chat_prompt | chat
response = chain.invoke({"text": "I love programming."})
""")

# =============================================================================
# 다른 페르소나 예제
# =============================================================================
print("\n" + "=" * 60)
print("다른 페르소나 예제: 셰익스피어 번역기")
print("=" * 60)

# 셰익스피어 스타일 번역기
shakespeare_template = "You are a helpful assistant that translates modern English to Shakespearean English."
shakespeare_system = SystemMessagePromptTemplate.from_template(shakespeare_template)

shakespeare_example_human = HumanMessagePromptTemplate.from_template("Hello")
shakespeare_example_ai = AIMessagePromptTemplate.from_template("Hail and well met!")

shakespeare_prompt = ChatPromptTemplate.from_messages([
    shakespeare_system,
    shakespeare_example_human,
    shakespeare_example_ai,
    human_message_prompt
])

shakespeare_chain = shakespeare_prompt | chat

try:
    print("셰익스피어 번역기 결과:")
    print("-" * 40)
    
    for sentence in test_sentences[:3]:  # 처음 3개만 테스트
        response = shakespeare_chain.invoke({"text": sentence})
        print(f"현대어: {sentence}")
        print(f"셰익스피어어: {response.content}")
        print()
        
except Exception as e:
    print(f"셰익스피어 번역기 오류: {e}")

# =============================================================================
# 실용적인 활용 예제
# =============================================================================
print("=" * 60)
print("실용적인 활용 예제: 정중한 이메일 변환기")
print("=" * 60)

# 정중한 이메일 변환기
email_template = "You are a helpful assistant that converts casual messages into polite, professional emails."
email_system = SystemMessagePromptTemplate.from_template(email_template)

email_example_human = HumanMessagePromptTemplate.from_template("Can you send me the report?")
email_example_ai = AIMessagePromptTemplate.from_template("Dear [Name],\n\nI hope this email finds you well. Would it be possible for you to send me the report when you have a moment?\n\nThank you for your time and assistance.\n\nBest regards,\n[Your Name]")

email_prompt = ChatPromptTemplate.from_messages([
    email_system,
    email_example_human,
    email_example_ai,
    human_message_prompt
])

email_chain = email_prompt | chat

casual_messages = [
    "Hey, where's my stuff?",
    "Need the files ASAP",
    "Can you fix this bug?"
]

try:
    print("정중한 이메일 변환 결과:")
    print("-" * 40)
    
    for message in casual_messages:
        response = email_chain.invoke({"text": message})
        print(f"캐주얼한 메시지: {message}")
        print(f"정중한 이메일:\n{response.content}")
        print("=" * 40)
        
except Exception as e:
    print(f"이메일 변환기 오류: {e}")

# =============================================================================
# 핵심 포인트 정리
# =============================================================================
print("\n" + "=" * 60)
print("핵심 포인트 정리")
print("=" * 60)

print("""
ChatPromptTemplate의 장점:
1. 구조화된 대화: 시스템-예시-사용자 메시지 순서로 명확한 구조
2. 역할 기반 설계: AI의 페르소나와 행동 방식을 명확히 정의
3. Few-shot Learning: 예시를 통해 원하는 응답 스타일 학습
4. 재사용성: 템플릿을 다양한 상황에 재활용 가능

실제 활용 분야:
- 언어 번역 및 스타일 변환
- 고객 서비스 챗봇
- 콘텐츠 톤 변경 (캐주얼 ↔ 정중함)
- 창의적 글쓰기 도우미

주의사항:
- 예시의 품질이 결과에 큰 영향을 미침
- 시스템 메시지는 명확하고 구체적으로 작성
- 과도한 예시는 토큰 사용량 증가의 원인
""")

print("\n7.2.1 ChatPromptTemplate 예제 완료!")


기존 환경 변수의 API 키를 사용합니다.
7.2.1 ChatPromptTemplate을 이용한 해적 번역기

원서 코드의 문제점:
1. LLMChain 클래스가 deprecated됨
2. chain.run() 메소드가 deprecated됨

수정 내용:
- LLMChain → RunnableSequence (prompt | llm)
- chain.run() → chain.invoke()

핵심 개념 - ChatPromptTemplate:
- 시스템 메시지: AI의 역할과 행동 방식 정의
- 예시 메시지: few-shot learning을 위한 대화 예시
- 인간 메시지: 사용자 입력 템플릿


생성된 프롬프트 구조:
1. SystemMessage: You are a helpful assistant that translates English to pirate.
2. HumanMessage: Hi
3. AIMessage: Argh me mateys
4. HumanMessage: I love programming.

실행 결과
해적 번역기 결과:
----------------------------------------
영어: I love programming.
해적어: I love plunderin' the code, arrr!

영어: How are you today?
해적어: How be ye doin' today, me hearty?

영어: The weather is beautiful.
해적어: Arrr, the weather be fair and fine, me hearties.

영어: Can you help me with my homework?
해적어: Aye, me heartie! I be happy to help ye with yer homework. What be ye needin' help with?

영어: Good morning!
해적어: Ahoy! Mornin' to ye!

기존 방식 vs 새로운 방식

원서 코드 (Deprecated)